 

# Comprehensive Handout: Stock Span Problem Using a Stack

## 1. IP–OP–PS (Input, Output, Problem Statement)

### **Problem Statement**

Given the daily stock prices of a company for \(n\) consecutive days, you need to compute the “span” of the stock’s price for each day. The span for a particular day is defined as the number of consecutive days (up to and including that day) for which the stock’s price was **less than or equal** to its price on the current day.

### **Input**

1. An integer \(n\) (the number of days).
2. An array of \(n\) integers, where each integer represents the stock price on that day.

### **Output**

- An array of \(n\) integers, where the \(i\)-th integer is the span of the stock on day \(i\).

### **Detailed Example**

**Input:**

```
n = 7
prices = [100, 80, 60, 70, 60, 75, 85]
```

**Output:**

```
span = [1, 1, 1, 2, 1, 4, 6]
```

**Explanation:**

1. **Day 0 (price = 100):** No previous days → span = 1  
2. **Day 1 (price = 80):** Previous day was 100 (which is higher), so only current day counts → span = 1  
3. **Day 2 (price = 60):** Previous days are [100, 80], both higher → span = 1  
4. **Day 3 (price = 70):** Previous days are [100, 80, 60]; the first smaller or equal price from the left is `60`, so the span is the distance from day 2 to day 3 plus 1 → span = 2  
5. **Day 4 (price = 60):** Previous day (day 3) had 70, which is higher → span = 1  
6. **Day 5 (price = 75):** Previous days [100, 80, 60, 70, 60], the nearest lower/equal streak includes day 4 (price=60), day 3 (price=70), day 2 (price=60), so total consecutive days = 4 → span = 4  
7. **Day 6 (price = 85):** Looking backward, 75, 60, 70, 60, 80, 100 are all less or equal until day 0’s 100 is not less or equal. So consecutive days = 6 → span = 6  

---

## 2. Identification

### **Why is this a Candidate for the Stack Technique?**

1. **Naive Approach (O(n²)):**  
   - For each day, you look backward until you find a day with a higher price. This can result in nested loops (for day \(i\), check days \(i-1, i-2, \dots\)).

2. **Key Cues / Characteristics:**
   - We only need to look back until we find a day with a higher price than the current day’s price.
   - The consecutive “span” suggests that once a price is overshadowed by a higher price, it is no longer relevant for future days.
   - A *monotonic stack* (descending) can efficiently skip the days that are no longer relevant.

3. **Stack Advantage (O(n)):**  
   - Each day is pushed and popped at most once.  
   - While the top of the stack has a price **less than or equal** to the current day’s price, pop it.  
   - The stack’s top will then be the nearest day with a higher price.

---

## 3. Break Down → Stack Approach

### **Step-by-Step Sub-Tasks**

1. **Initialization:**
   - Create an empty stack to store **indices** of days.
   - Create an output array `span` of size \(n\).

2. **Iterate Over Days (Left to Right):**
   - For the current day \(i\):
     1. **Pop** from the stack while it is not empty **and** the price at the top of the stack is **less than or equal** to the current day’s price.  
     2. If the stack is **empty**, it means no previous day’s price is higher → `span[i] = i + 1`.  
     3. Otherwise, the top of the stack is the index of the nearest higher price on the left → `span[i] = i - topIndex`.  
     4. **Push** the current index \(i\) onto the stack.

3. **Data Structures Used:**
   - **Stack of Indices**:  
     - The stack will store the indices of days in descending order of prices.
   - **Array for Span**:  
     - Stores the final span result for each day.

---

## 4. Explanations + Code

### **Detailed Explanation of Each Step**

1. **Stack Maintenance:**
   - By maintaining a *descending order* of prices on the stack, we can quickly pop out any days whose prices are not relevant (i.e., less than or equal to current price).
2. **Computing Span:**
   - If the stack becomes empty, it means no higher price was found → the span extends all the way back to the first day.
   - If the stack is not empty, the nearest higher day is on the top. The difference in indices gives the span.
3. **Time Complexity:**
   - Each day (index) is pushed and popped **at most once**.
   - Hence, the overall time complexity is **O(n)**.

### **C++ Implementation**

```cpp
#include <iostream>
#include <vector>
#include <stack>
using namespace std;

vector<int> calculateSpan(const vector<int> &prices) {
    int n = prices.size();
    vector<int> span(n, 1);
    stack<int> st;  // stack will hold indices

    // The first day has a span of 1
    // We'll handle it in the loop, but we can also do st.push(0) first.

    for(int i = 0; i < n; i++) {
        // Pop while the stack is not empty and the price at st.top() <= current price
        while(!st.empty() && prices[st.top()] <= prices[i]) {
            st.pop();
        }

        // If the stack is empty, no previous higher price
        if(st.empty()) {
            span[i] = i + 1;
        } else {
            // The top of the stack is the index of the previous higher price
            span[i] = i - st.top();
        }
        // Push the current day index onto the stack
        st.push(i);
    }
    return span;
}

int main() {
    vector<int> prices = {100, 80, 60, 70, 60, 75, 85};
    vector<int> result = calculateSpan(prices);

    cout << "Prices: ";
    for(auto p : prices) cout << p << " ";
    cout << "\nSpan:   ";
    for(auto r : result) cout << r << " ";
    cout << endl;

    return 0;
}
```

---

## 5. Animated Visualization

Below is a **Python** code snippet using **matplotlib** and **ipywidgets** that illustrates the **internal state** of the algorithm at each step. You can copy this into a Jupyter Notebook cell, run it, and then move a slider to see how the stack evolves day by day.

```python
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

def stock_span_visual(step_index):
    prices = [100, 80, 60, 70, 60, 75, 85]
    n = len(prices)
    span = [0]*n
    st = []
    states = []

    for i in range(n):
        state_info = {
            'current_index': i,
            'current_price': prices[i],
            'stack_before': st.copy(),
            'popped': [],
            'span_so_far': span.copy()
        }

        # Pop while top is <= current price
        popped_indices = []
        while st and prices[st[-1]] <= prices[i]:
            popped_indices.append(st.pop())

        state_info['popped'] = popped_indices

        # If stack is empty, span is i+1
        if not st:
            span[i] = i + 1
        else:
            span[i] = i - st[-1]

        st.append(i)
        state_info['stack_after'] = st.copy()
        state_info['span_so_far'] = span.copy()
        states.append(state_info)

    # Limit step_index to valid range
    if step_index >= len(states):
        step_index = len(states) - 1

    step = states[step_index]

    fig, ax = plt.subplots(figsize=(8,4))
    ax.set_title(f"Step {step_index + 1}: Processing index {step['current_index']} (Price: {step['current_price']})")
    ax.set_xlabel("Day Index")
    ax.set_ylabel("Price")

    # Plot the prices
    ax.bar(range(n), prices, color='skyblue')
    # Highlight the current day's bar in orange
    ax.bar(step['current_index'], prices[step['current_index']], color='orange')

    # Show stack changes
    annotation = (f"Stack before: {step['stack_before']}\n"
                  f"Popped indices: {step['popped']}\n"
                  f"Stack after: {step['stack_after']}\n"
                  f"Span so far: {step['span_so_far']}")
    ax.text(0.02, 0.95, annotation, transform=ax.transAxes, fontsize=10,
            bbox=dict(facecolor='white', alpha=0.7), verticalalignment='top')

    plt.ylim(0, max(prices) + 10)
    plt.show()

interact(stock_span_visual, step_index=IntSlider(min=0, max=len(prices)-1, step=1, value=0));
```

### **How to Use This Visualization**
1. Copy the snippet into a Jupyter Notebook cell.
2. Run the cell.
3. Use the slider labeled `step_index` to move through each iteration.
4. Observe:
   - **Stack before** the current iteration.
   - **Popped indices** (those that are less than or equal to the current price).
   - **Stack after** the iteration.
   - **Span so far** shows the partial results up to that point.

---

# Summary

1. **Problem Statement (IP–OP–PS):**  
   - Compute the span for each day’s stock price.  

2. **Identification:**  
   - Brute force is O(n²). A stack-based solution avoids redundant comparisons, reducing complexity to O(n).  

3. **Breakdown (Stack):**  
   - Maintain a descending stack of indices. Pop indices whose prices are <= current price. The difference in indices gives the span.  

4. **Explanations + Code:**  
   - Each element is pushed/popped at most once → O(n).  
   - A full C++ solution is provided.  

5. **Animated Visualization:**  
   - The Python code snippet (matplotlib + ipywidgets) demonstrates each step of the algorithm, helping you understand how the stack evolves.  
 